In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import os
import requests
import hvplot.pandas
from pathlib import Path
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_blobs,make_moons
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from sklearn.metrics import r2_score
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read in any csv files
funds_df = pd.read_csv(Path('bill-rates-2002-2023.csv'),
                           index_col='Date',
                           parse_dates=True,
                      infer_datetime_format=True)
funds_df = funds_df.dropna()   
funds_df

,4 WEEKS BANK DISCOUNT,4 WEEKS COUPON EQUIVALENT,8 WEEKS BANK DISCOUNT,8 WEEKS COUPON EQUIVALENT,13 WEEKS BANK DISCOUNT,13 WEEKS COUPON EQUIVALENT,17 WEEKS BANK DISCOUNT,17 WEEKS COUPON EQUIVALENT,26 WEEKS BANK DISCOUNT,26 WEEKS COUPON EQUIVALENT,52 WEEKS BANK DISCOUNT,52 WEEKS COUPON EQUIVALENT
Date,,,,,,,,,,,,
2023-12-29,5.33,5.44,5.33,5.46,5.20,5.35,5.21,5.39,5.05,5.26,4.55,4.79
2023-12-28,5.31,5.42,5.29,5.42,5.25,5.41,5.22,5.40,5.06,5.28,4.58,4.82
2023-12-27,5.28,5.39,5.29,5.42,5.24,5.40,5.21,5.39,5.05,5.27,4.54,4.78
2023-12-26,5.27,5.38,5.27,5.40,5.25,5.41,5.22,5.40,5.06,5.28,4.58,4.82
2023-12-22,5.27,5.38,5.26,5.39,5.24,5.39,5.22,5.40,5.10,5.32,4.62,4.86
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,3.40,3.46,3.62,3.69,3.97,4.07,4.12,4.23,4.36,4.52,4.40,4.61
2022-10-24,3.42,3.48,3.65,3.72,4.00,4.10,4.14,4.26,4.38,4.54,4.40,4.61
2022-10-21,3.42,3.48,3.64,3.71,3.90,3.99,4.13,4.25,4.29,4.44,4.38,4.59


In [3]:
fed_funds_graph = funds_df.hvplot(title='Federal Funds Rate')
spy_close_graph = ticker_close['SPY Close'].hvplot()

NameError: name 'ticker_close' is not defined

In [ ]:
funds_spy_compare = spy_close_graph + fed_funds_graph
funds_spy_compare

In [4]:
# Import API data
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [9]:
#3
# Load .env enviroment variables/
load_dotenv('ProjectAPI.env')

True

In [10]:
#4
# Set the tickers
tickers = ['SPY','EWA','EWC','EWJ','EWU','EWZ','EWW','EZU','IWM','MCHI']
# Set potential feature tickers
#tickers_2 = ['BABA','SAP','
# Create the shares DataFrame
df_etf = pd.DataFrame(index=tickers)
#alt_df_etf = pd.DataFrame(index=tickers_2)

In [15]:
#5
# Set Alpaca API key and secret
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [16]:
#6
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [17]:
#6a
# Create current date variable
import datetime
today = datetime.date.today()
historical = today - datetime.timedelta(days=550)

In [18]:
#6b
# Format current date as ISO format
# Set start and end datetimes of 18 months
start_date = pd.Timestamp(historical, tz="America/New_York").isoformat()
end_date = pd.Timestamp(today, tz="America/New_York").isoformat()

In [19]:
#7
# Set timeframe to 18 months for the Alpaca API
timeframe = "1Day"


In [20]:
#8
# Get 18 months of pricing data for ETF portfolio
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date).df

df_portfolio

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2022-08-25 04:00:00+00:00,22.87,22.87,22.660,4952,22.71,3913938,22.766495,EWA
2022-08-26 04:00:00+00:00,22.44,23.14,22.410,9323,23.11,4546397,22.738378,EWA
2022-08-29 04:00:00+00:00,22.36,22.46,22.265,8311,22.35,3117618,22.380320,EWA
2022-08-30 04:00:00+00:00,22.05,22.49,22.010,9813,22.49,4363121,22.153321,EWA
2022-08-31 04:00:00+00:00,21.91,22.19,21.910,8650,22.14,3948736,22.029738,EWA
...,...,...,...,...,...,...,...,...
2024-02-20 05:00:00+00:00,496.76,498.41,494.450,595486,497.72,71736740,496.441094,SPY
2024-02-21 05:00:00+00:00,497.21,497.37,493.560,499405,495.42,59603771,495.831166,SPY
2024-02-22 05:00:00+00:00,507.50,508.49,503.020,613961,504.01,76402535,505.832024,SPY


In [21]:
# Drop time from index
df_portfolio.index = df_portfolio.index.date
df_portfolio

,close,high,low,trade_count,open,volume,vwap,symbol
2022-08-25,22.87,22.87,22.660,4952,22.71,3913938,22.766495,EWA
2022-08-26,22.44,23.14,22.410,9323,23.11,4546397,22.738378,EWA
2022-08-29,22.36,22.46,22.265,8311,22.35,3117618,22.380320,EWA
2022-08-30,22.05,22.49,22.010,9813,22.49,4363121,22.153321,EWA
2022-08-31,21.91,22.19,21.910,8650,22.14,3948736,22.029738,EWA
...,...,...,...,...,...,...,...,...
2024-02-20,496.76,498.41,494.450,595486,497.72,71736740,496.441094,SPY
2024-02-21,497.21,497.37,493.560,499405,495.42,59603771,495.831166,SPY
2024-02-22,507.50,508.49,503.020,613961,504.01,76402535,505.832024,SPY
2024-02-23,507.85,510.13,507.100,489047,509.27,61309016,508.438618,SPY


In [22]:
# Drop time from index
historical_df.index = historical_df.index.date
y_historical_df.index = y_historical_df.index.date

NameError: name 'historical_df' is not defined

In [23]:
corr_df['SPY Close']

NameError: name 'corr_df' is not defined

In [24]:
# Separate ticker data
SPY = df_portfolio[df_portfolio['symbol']=='SPY'].drop('symbol',axis=1)
EWA = df_portfolio[df_portfolio['symbol']=='EWA'].drop('symbol',axis=1)
EWC = df_portfolio[df_portfolio['symbol']=='EWC'].drop('symbol',axis=1)
EWJ = df_portfolio[df_portfolio['symbol']=='EWJ'].drop('symbol',axis=1)
EWU = df_portfolio[df_portfolio['symbol']=='EWU'].drop('symbol',axis=1)
EWZ = df_portfolio[df_portfolio['symbol']=='EWZ'].drop('symbol',axis=1)
EWW = df_portfolio[df_portfolio['symbol']=='EWW'].drop('symbol',axis=1)
EZU = df_portfolio[df_portfolio['symbol']=='EZU'].drop('symbol',axis=1)
IWM = df_portfolio[df_portfolio['symbol']=='IWM'].drop('symbol',axis=1)
MCHI = df_portfolio[df_portfolio['symbol']=='MCHI'].drop('symbol',axis=1)

In [25]:
# Concatenate tickers
ticker_portfolio = pd.concat([
    SPY,EWA,EWC,EWJ,EWU,EWZ,EWW,EZU,IWM,MCHI],axis=1,keys=[
    'SPY','EWA','EWC','EWJ','EWU','EWZ','EWW','EZU','IWM','MCHI'])
ticker_portfolio                                                           

SPY                                                   \
             close    high       low trade_count    open     volume   
2022-08-25  419.51  419.56  414.0900      339152  415.24   51062473   
2022-08-26  405.31  419.96  405.2500      669022  419.39  105714602   
2022-08-29  402.63  405.84  401.1999      490122  402.20   66499372   
2022-08-30  398.21  404.10  396.0000      610728  403.85   87190352   
2022-08-31  395.18  401.24  395.0400      504194  399.93   77635142   
...            ...     ...       ...         ...     ...        ...   
2024-02-20  496.76  498.41  494.4500      595486  497.72   71736740   
2024-02-21  497.21  497.37  493.5600      499405  495.42   59603771   
2024-02-22  507.50  508.49  503.0200      613961  504.01   76402535   
2024-02-23  507.85  510.13  507.1000      489047  509.27   61309016   
2024-02-26  505.99  508.75  505.8600      434433  508.30   50386887   

                          EWA                    ...     IWM            \
                  vwap  close     high      low  ...    open    volume   
2022-08-25  416.913991  22.87  22.8700  22.6600  ...  193.38  15276937   
2022-08-26  410.671909  22.44  23.1400  22.4100  ...  195.34  24682950   
2022-08-29  403.536027  22.36  22.4600  22.2650  ...  186.95  20015752   
2022-08-30  399.076591  22.05  22.4900  22.0100  ...  187.82  22928104   
2022-08-31  397.187287  21.91  22.1900  21.9100  ...  184.94  22457303   
...                ...    ...      ...      ...  ...     ...       ...   
2024-02-20  496.441094  23.75  23.9099  23.6797  ...  199.15  30547481   
2024-02-21  495.831166  23.63  23.6900  23.5200  ...  197.66  28821470   
2024-02-22  505.832024  23.86  23.8750  23.7105  ...  198.65  38184703   
2024-02-23  508.438618  23.91  23.9679  23.8700  ...  199.60  37656936   
2024-02-26  507.079425  23.83  23.9050  23.7550  ...  199.69  29118414   

                         MCHI                                                \
                  vwap  close     high     low trade_count    open   volume   
2022-08-25  194.489459  50.79  50.8100  49.620       34324  49.840  4362289   
2022-08-26  191.039871  50.38  51.9300  50.300       48695  51.815  8188776   
2022-08-29  187.767002  49.95  50.8650  49.930       48971  50.270  7306766   
2022-08-30  185.039434  48.79  49.7600  48.585       33010  49.650  5234435   
2022-08-31  184.087820  49.69  50.3250  49.530       35040  49.790  6156630   
...                ...    ...      ...     ...         ...     ...      ...   
2024-02-20  198.936731  38.47  38.8700  38.280       20257  38.780  2806153   
2024-02-21  197.569559  39.28  39.6150  39.120       18689  39.290  3192627   
2024-02-22  199.066619  39.77  39.8700  39.455       21611  39.810  3510076   
2024-02-23  199.850984  39.90  40.1900  39.730       20980  40.110  6138575   
2024-02-26  200.940152  39.55  39.8607  39.540       24944  39.670  7433109   

                       
                 vwap  
2022-08-25  50.434104  
2022-08-26  50.675660  
2022-08-29  50.371550  
2022-08-30  48.894582  
2022-08-31  49.787465  
...               ...  
2024-02-20  38.495465  
2024-02-21  39.293091  
2024-02-22  39.681310  
2024-02-23  39.897140  
2024-02-26  39.616831  

[377 rows x 70 columns]

In [26]:
# Create a dataframe for closing prices
ticker_close = pd.DataFrame()
ticker_close['SPY Close'] = ticker_portfolio['SPY']['close']
ticker_close['EWA Close'] = ticker_portfolio['EWA']['close']
ticker_close['EWC Close'] = ticker_portfolio['EWC']['close']
ticker_close['EWJ Close'] = ticker_portfolio['EWJ']['close']
ticker_close['EWU Close'] = ticker_portfolio['EWU']['close']
ticker_close['EWZ Close'] = ticker_portfolio['EWZ']['close']
ticker_close['EWW Close'] = ticker_portfolio['EWW']['close']
ticker_close['EZU Close'] = ticker_portfolio['EZU']['close']
ticker_close['IWM Close'] = ticker_portfolio['EWA']['close']
ticker_close['MCHI Close'] = ticker_portfolio['MCHI']['close']
ticker_close.shape

(377, 10)

In [27]:
# Create dataframe for training on historical data (X features)
historical_df = ticker_close['SPY Close']
historical_df

2022-08-25    419.51
2022-08-26    405.31
2022-08-29    402.63
2022-08-30    398.21
2022-08-31    395.18
               ...  
2024-02-20    496.76
2024-02-21    497.21
2024-02-22    507.50
2024-02-23    507.85
2024-02-26    505.99
Name: SPY Close, Length: 377, dtype: float64

In [28]:
# Determine correlation between SPY and ETFs
ticker_close[[
    'SPY Close','EWA Close','EWC Close','EWJ Close','EWU Close',
    'EWZ Close','EWW Close', 'EZU Close','IWM Close','MCHI Close']].corr().style.background_gradient()


,SPY Close,EWA Close,EWC Close,EWJ Close,EWU Close,EWZ Close,EWW Close,EZU Close,IWM Close,MCHI Close
SPY Close,1.000000,0.448551,0.723053,0.937458,0.608277,0.629655,0.862108,0.761973,0.448551,-0.449946
EWA Close,0.448551,1.000000,0.863272,0.545639,0.774027,0.042876,0.602138,0.725026,1.000000,0.333255
EWC Close,0.723053,0.863272,1.000000,0.722426,0.744928,0.388883,0.740772,0.757639,0.863272,0.004287
EWJ Close,0.937458,0.545639,0.722426,1.000000,0.781705,0.438663,0.928945,0.891757,0.545639,-0.253411
EWU Close,0.608277,0.774027,0.744928,0.781705,1.000000,0.010681,0.823148,0.947689,0.774027,0.239815
EWZ Close,0.629655,0.042876,0.388883,0.438663,0.010681,1.000000,0.403209,0.162010,0.042876,-0.605152
EWW Close,0.862108,0.602138,0.740772,0.928945,0.823148,0.403209,1.000000,0.941962,0.602138,-0.219472
EZU Close,0.761973,0.725026,0.757639,0.891757,0.947689,0.162010,0.941962,1.000000,0.725026,0.034025
IWM Close,0.448551,1.000000,0.863272,0.545639,0.774027,0.042876,0.602138,0.725026,1.000000,0.333255
MCHI Close,-0.449946,0.333255,0.004287,-0.253411,0.239815,-0.605152,-0.219472,0.034025,0.333255,1.000000


In [29]:
# View ticker correlation in dataframe
corr_df = ticker_close[[
    'SPY Close','EWA Close','EWC Close','EWJ Close','EWU Close',
    'EWZ Close','EWW Close', 'EZU Close','IWM Close','MCHI Close']].corr()
print(corr_df)

            SPY Close  EWA Close  EWC Close  EWJ Close  EWU Close  EWZ Close  \
SPY Close    1.000000   0.448551   0.723053   0.937458   0.608277   0.629655   
EWA Close    0.448551   1.000000   0.863272   0.545639   0.774027   0.042876   
EWC Close    0.723053   0.863272   1.000000   0.722426   0.744928   0.388883   
EWJ Close    0.937458   0.545639   0.722426   1.000000   0.781705   0.438663   
EWU Close    0.608277   0.774027   0.744928   0.781705   1.000000   0.010681   
EWZ Close    0.629655   0.042876   0.388883   0.438663   0.010681   1.000000   
EWW Close    0.862108   0.602138   0.740772   0.928945   0.823148   0.403209   
EZU Close    0.761973   0.725026   0.757639   0.891757   0.947689   0.162010   
IWM Close    0.448551   1.000000   0.863272   0.545639   0.774027   0.042876   
MCHI Close  -0.449946   0.333255   0.004287  -0.253411   0.239815  -0.605152   

            EWW Close  EZU Close  IWM Close  MCHI Close  
SPY Close    0.862108   0.761973   0.448551   -0.449946  
EWA

In [30]:
# Concatenate SPY Close and Fed Funds dataframes
funds_spy_df = pd.concat([ticker_close['SPY Close'],funds_df], axis=1)
funds_spy_df = funds_spy_df.dropna()
funds_spy_df.head()

,SPY Close,4 WEEKS BANK DISCOUNT,4 WEEKS COUPON EQUIVALENT,8 WEEKS BANK DISCOUNT,8 WEEKS COUPON EQUIVALENT,13 WEEKS BANK DISCOUNT,13 WEEKS COUPON EQUIVALENT,17 WEEKS BANK DISCOUNT,17 WEEKS COUPON EQUIVALENT,26 WEEKS BANK DISCOUNT,26 WEEKS COUPON EQUIVALENT,52 WEEKS BANK DISCOUNT,52 WEEKS COUPON EQUIVALENT
2022-10-19,368.50,3.13,3.18,3.52,3.59,3.91,4.00,4.15,4.27,4.31,4.47,4.39,4.60
2022-10-20,365.41,3.44,3.50,3.70,3.77,3.91,4.00,4.15,4.27,4.34,4.50,4.44,4.65
2022-10-21,374.29,3.42,3.48,3.64,3.71,3.90,3.99,4.13,4.25,4.29,4.44,4.38,4.59
2022-10-24,378.87,3.42,3.48,3.65,3.72,4.00,4.10,4.14,4.26,4.38,4.54,4.40,4.61
2022-10-25,384.92,3.40,3.46,3.62,3.69,3.97,4.07,4.12,4.23,4.36,4.52,4.40,4.61


In [31]:
# Drop unnecessary columns
funds_spy_df_2 = funds_spy_df.drop(columns=['4 WEEKS BANK DISCOUNT','4 WEEKS COUPON EQUIVALENT',
                                           '8 WEEKS BANK DISCOUNT', '8 WEEKS COUPON EQUIVALENT',
                                           '13 WEEKS BANK DISCOUNT', '13 WEEKS COUPON EQUIVALENT',
                                           '17 WEEKS BANK DISCOUNT', '17 WEEKS COUPON EQUIVALENT',
                                           '26 WEEKS BANK DISCOUNT', '26 WEEKS COUPON EQUIVALENT', 
                                            '52 WEEKS COUPON EQUIVALENT'])
funds_spy_df_2

,SPY Close,52 WEEKS BANK DISCOUNT
2022-10-19,368.50,4.39
2022-10-20,365.41,4.44
2022-10-21,374.29,4.38
2022-10-24,378.87,4.40
2022-10-25,384.92,4.40
...,...,...
2023-12-22,473.65,4.62
2023-12-26,475.65,4.58
2023-12-27,476.51,4.54
2023-12-28,476.69,4.58


In [32]:
# Rename '52 WEEKS' column
funds_spy_df_2['Fed Interest Rates'] = funds_spy_df_2['52 WEEKS BANK DISCOUNT']
funds_spy_df_2 = funds_spy_df_2.drop(columns=['52 WEEKS BANK DISCOUNT'])
funds_spy_df_2

,SPY Close,Fed Interest Rates
2022-10-19,368.50,4.39
2022-10-20,365.41,4.44
2022-10-21,374.29,4.38
2022-10-24,378.87,4.40
2022-10-25,384.92,4.40
...,...,...
2023-12-22,473.65,4.62
2023-12-26,475.65,4.58
2023-12-27,476.51,4.54
2023-12-28,476.69,4.58


In [33]:
# Visualize and compare interest rate and closing price data
funds_spy_df_2.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [34]:
# Determine correlation between federal interest rates and SPY Closing price
funds_spy_corr = funds_spy_df_2.corr()
funds_spy_corr['SPY Close'].iloc[1:]

Fed Interest Rates    0.67872
Name: SPY Close, dtype: float64

In [35]:
funds_spy_corr

,SPY Close,Fed Interest Rates
SPY Close,1.00000,0.67872
Fed Interest Rates,0.67872,1.00000


In [36]:
# Create a list to hold correlated etfs and respective symbols
# Create for loop to append highest correlated etfs in respective list
correlated_etfs = []
symbols = []
for index,df in corr_df.iterrows():
    if index == 'SPY Close':
        continue 
    if df['SPY Close'] >= .67:
        correlated_etfs.append(df['SPY Close'])
        symbols.append(index)
       
print(correlated_etfs)
print(symbols)

[0.7230527403462677, 0.9374576660716222, 0.8621081178698599, 0.7619728375717485]
['EWC Close', 'EWJ Close', 'EWW Close', 'EZU Close']


In [37]:
symbol_df

NameError: name 'symbol_df' is not defined

In [38]:
# Set trading parameters
symbol_df = ticker_close[symbols].copy() 
symbol_df['EWJ Returns'] = symbol_df['EWJ Close'].pct_change()
symbol_df['EWW Returns'] = symbol_df['EWW Close'].pct_change()
symbol_df['EZU Returns'] = symbol_df['EZU Close'].pct_change()
symbol_df['EWC Returns'] = symbol_df['EWC Close'].pct_change()
symbol_df = symbol_df.dropna()

#symbol_df.loc[symbol_df['EWJ Returns'] > .005,'y'] = 1
symbol_df

,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns
2022-08-26,34.96,54.44,46.54,35.59,-0.024373,-0.022885,-0.033931,-0.024009
2022-08-29,34.98,53.89,45.92,35.71,-0.010103,-0.013322,0.003372,0.000572
2022-08-30,34.13,53.83,45.17,35.60,-0.001113,-0.016333,-0.003080,-0.024300
2022-08-31,33.69,53.59,44.25,35.33,-0.004458,-0.020368,-0.007584,-0.012892
2022-09-01,33.36,52.95,44.68,34.80,-0.011943,0.009718,-0.015001,-0.009795
...,...,...,...,...,...,...,...,...
2024-02-20,36.54,67.93,67.31,48.06,0.004436,0.000149,0.005860,-0.003817
2024-02-21,36.49,67.79,67.12,48.31,-0.002061,-0.002823,0.005202,-0.001368
2024-02-22,36.84,68.90,67.20,48.98,0.016374,0.001192,0.013869,0.009592
2024-02-23,36.99,69.01,66.53,49.01,0.001597,-0.009970,0.000612,0.004072


In [39]:
# Save a new copy of symbol_df into features_df and begin adding more features
features_df = symbol_df.copy()
features_df

,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns
2022-08-26,34.96,54.44,46.54,35.59,-0.024373,-0.022885,-0.033931,-0.024009
2022-08-29,34.98,53.89,45.92,35.71,-0.010103,-0.013322,0.003372,0.000572
2022-08-30,34.13,53.83,45.17,35.60,-0.001113,-0.016333,-0.003080,-0.024300
2022-08-31,33.69,53.59,44.25,35.33,-0.004458,-0.020368,-0.007584,-0.012892
2022-09-01,33.36,52.95,44.68,34.80,-0.011943,0.009718,-0.015001,-0.009795
...,...,...,...,...,...,...,...,...
2024-02-20,36.54,67.93,67.31,48.06,0.004436,0.000149,0.005860,-0.003817
2024-02-21,36.49,67.79,67.12,48.31,-0.002061,-0.002823,0.005202,-0.001368
2024-02-22,36.84,68.90,67.20,48.98,0.016374,0.001192,0.013869,0.009592
2024-02-23,36.99,69.01,66.53,49.01,0.001597,-0.009970,0.000612,0.004072


In [40]:
# Add SPY closing price daily returns to features dataframe
features_df['SPY Returns'] = ticker_close['SPY Close'].pct_change()

features_df

,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns,SPY Returns
2022-08-26,34.96,54.44,46.54,35.59,-0.024373,-0.022885,-0.033931,-0.024009,-0.033849
2022-08-29,34.98,53.89,45.92,35.71,-0.010103,-0.013322,0.003372,0.000572,-0.006612
2022-08-30,34.13,53.83,45.17,35.60,-0.001113,-0.016333,-0.003080,-0.024300,-0.010978
2022-08-31,33.69,53.59,44.25,35.33,-0.004458,-0.020368,-0.007584,-0.012892,-0.007609
2022-09-01,33.36,52.95,44.68,34.80,-0.011943,0.009718,-0.015001,-0.009795,0.003138
...,...,...,...,...,...,...,...,...,...
2024-02-20,36.54,67.93,67.31,48.06,0.004436,0.000149,0.005860,-0.003817,-0.005505
2024-02-21,36.49,67.79,67.12,48.31,-0.002061,-0.002823,0.005202,-0.001368,0.000906
2024-02-22,36.84,68.90,67.20,48.98,0.016374,0.001192,0.013869,0.009592,0.020695
2024-02-23,36.99,69.01,66.53,49.01,0.001597,-0.009970,0.000612,0.004072,0.000690


In [41]:
features_df['y (SPY Close)'] = historical_df
features_df

,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns,SPY Returns,y (SPY Close)
2022-08-26,34.96,54.44,46.54,35.59,-0.024373,-0.022885,-0.033931,-0.024009,-0.033849,405.31
2022-08-29,34.98,53.89,45.92,35.71,-0.010103,-0.013322,0.003372,0.000572,-0.006612,402.63
2022-08-30,34.13,53.83,45.17,35.60,-0.001113,-0.016333,-0.003080,-0.024300,-0.010978,398.21
2022-08-31,33.69,53.59,44.25,35.33,-0.004458,-0.020368,-0.007584,-0.012892,-0.007609,395.18
2022-09-01,33.36,52.95,44.68,34.80,-0.011943,0.009718,-0.015001,-0.009795,0.003138,396.42
...,...,...,...,...,...,...,...,...,...,...
2024-02-20,36.54,67.93,67.31,48.06,0.004436,0.000149,0.005860,-0.003817,-0.005505,496.76
2024-02-21,36.49,67.79,67.12,48.31,-0.002061,-0.002823,0.005202,-0.001368,0.000906,497.21
2024-02-22,36.84,68.90,67.20,48.98,0.016374,0.001192,0.013869,0.009592,0.020695,507.50
2024-02-23,36.99,69.01,66.53,49.01,0.001597,-0.009970,0.000612,0.004072,0.000690,507.85


In [42]:
# Rearrange columns
features_df = features_df.iloc[:,[0,1,2,3,9,4,5,6,7,8]]
features_df

,EWC Close,EWJ Close,EWW Close,EZU Close,y (SPY Close),EWJ Returns,EWW Returns,EZU Returns,EWC Returns,SPY Returns
2022-08-26,34.96,54.44,46.54,35.59,405.31,-0.024373,-0.022885,-0.033931,-0.024009,-0.033849
2022-08-29,34.98,53.89,45.92,35.71,402.63,-0.010103,-0.013322,0.003372,0.000572,-0.006612
2022-08-30,34.13,53.83,45.17,35.60,398.21,-0.001113,-0.016333,-0.003080,-0.024300,-0.010978
2022-08-31,33.69,53.59,44.25,35.33,395.18,-0.004458,-0.020368,-0.007584,-0.012892,-0.007609
2022-09-01,33.36,52.95,44.68,34.80,396.42,-0.011943,0.009718,-0.015001,-0.009795,0.003138
...,...,...,...,...,...,...,...,...,...,...
2024-02-20,36.54,67.93,67.31,48.06,496.76,0.004436,0.000149,0.005860,-0.003817,-0.005505
2024-02-21,36.49,67.79,67.12,48.31,497.21,-0.002061,-0.002823,0.005202,-0.001368,0.000906
2024-02-22,36.84,68.90,67.20,48.98,507.50,0.016374,0.001192,0.013869,0.009592,0.020695
2024-02-23,36.99,69.01,66.53,49.01,507.85,0.001597,-0.009970,0.000612,0.004072,0.000690


In [43]:
# Add "y" column (SPY historical closing price) for testing neural models
#features_df['y (Historical_SPY)'] = y_historical_df
features_df['y'] = 0
features_df = features_df.drop(columns=['y (SPY Close)'])
features_df

,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns,SPY Returns,y
2022-08-26,34.96,54.44,46.54,35.59,-0.024373,-0.022885,-0.033931,-0.024009,-0.033849,0
2022-08-29,34.98,53.89,45.92,35.71,-0.010103,-0.013322,0.003372,0.000572,-0.006612,0
2022-08-30,34.13,53.83,45.17,35.60,-0.001113,-0.016333,-0.003080,-0.024300,-0.010978,0
2022-08-31,33.69,53.59,44.25,35.33,-0.004458,-0.020368,-0.007584,-0.012892,-0.007609,0
2022-09-01,33.36,52.95,44.68,34.80,-0.011943,0.009718,-0.015001,-0.009795,0.003138,0
...,...,...,...,...,...,...,...,...,...,...
2024-02-20,36.54,67.93,67.31,48.06,0.004436,0.000149,0.005860,-0.003817,-0.005505,0
2024-02-21,36.49,67.79,67.12,48.31,-0.002061,-0.002823,0.005202,-0.001368,0.000906,0
2024-02-22,36.84,68.90,67.20,48.98,0.016374,0.001192,0.013869,0.009592,0.020695,0
2024-02-23,36.99,69.01,66.53,49.01,0.001597,-0.009970,0.000612,0.004072,0.000690,0


In [44]:
features_df.loc[features_df['SPY Returns'] > 0.005,'y'] = 1
features_df.loc[features_df['SPY Returns'] < -0.005,'y'] = -1
features_df



,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns,SPY Returns,y
2022-08-26,34.96,54.44,46.54,35.59,-0.024373,-0.022885,-0.033931,-0.024009,-0.033849,-1
2022-08-29,34.98,53.89,45.92,35.71,-0.010103,-0.013322,0.003372,0.000572,-0.006612,-1
2022-08-30,34.13,53.83,45.17,35.60,-0.001113,-0.016333,-0.003080,-0.024300,-0.010978,-1
2022-08-31,33.69,53.59,44.25,35.33,-0.004458,-0.020368,-0.007584,-0.012892,-0.007609,-1
2022-09-01,33.36,52.95,44.68,34.80,-0.011943,0.009718,-0.015001,-0.009795,0.003138,0
...,...,...,...,...,...,...,...,...,...,...
2024-02-20,36.54,67.93,67.31,48.06,0.004436,0.000149,0.005860,-0.003817,-0.005505,-1
2024-02-21,36.49,67.79,67.12,48.31,-0.002061,-0.002823,0.005202,-0.001368,0.000906,0
2024-02-22,36.84,68.90,67.20,48.98,0.016374,0.001192,0.013869,0.009592,0.020695,1
2024-02-23,36.99,69.01,66.53,49.01,0.001597,-0.009970,0.000612,0.004072,0.000690,0


In [ ]:
# NOT MEANINGFUL FOR NOW --------------------------------------------------
# Calculate average pct_change (returns) for etfs
ezu_avg_return = symbol_df['EZU Returns'].mean()
ewj_avg_return = symbol_df['EWJ Returns'].mean()
eww_avg_return = symbol_df['EWW Returns'].mean()
ewc_avg_return = symbol_df['EWC Returns'].mean()

In [45]:
etf_avg = pd.DataFrame(columns=['EZU avg','EWJ avg','EWW avg'],
                       data=[[ezu_avg_return,ewj_avg_return,eww_avg_return]])
etf_avg['EZU avg'] = ezu_avg_return
etf_avg['EWJ avg'] = ewj_avg_return
etf_avg['EWW avg'] = eww_avg_return
etf_avg

NameError: name 'ezu_avg_return' is not defined

In [ ]:
total = etf_avg.sum()
total

In [ ]:
# Determine overall average pct_change (returns) for etfs >= .75 correlation threshold
# **GET HELP ---- corr_true_return = correlated_etfs/len(correlated_etfs)
corr_avg = (ezu_avg_return + ewj_avg_return + eww_avg_return)/3
corr_avg

In [ ]:
for index,row in symbol_df.iterrows():
    row.loc[index,'sum'] = row.loc[:,0:2].sum()
   

In [46]:
corr_true_return = correlated_etfs.cumsum()/len(correlated_etfs)

AttributeError: 'list' object has no attribute 'cumsum'

In [ ]:
# ----------------------------------------------------------------------------------

In [ ]:
# PAUSE... NEURAL NETWORK MODELING BEGINS ------------------------------------
# ---------------------------------------------------------------------------- 
# ----------------------------------------------------------------------------        

In [ ]:
features_df

In [70]:
# Set X (these are the initial features for neural network)
# Create a shift (lag) to verify effect of correlation on SPY Returns 
""" EWJ, EWW, EZU columns are lagged by 1 day; therefore, if SPY behaves similar to
the correlated columns, this verifies that correlation can be used as a viable predicter
of future returns for the ETF.""" 

X = features_df[[
                 'EWJ Returns','EWW Returns','EZU Returns','EWC Returns',
                ]].shift().dropna().copy()
X.head(3)

,EWJ Returns,EWW Returns,EZU Returns,EWC Returns
2022-08-29,-0.024373,-0.022885,-0.033931,-0.024009
2022-08-30,-0.010103,-0.013322,0.003372,0.000572
2022-08-31,-0.001113,-0.016333,-0.003080,-0.024300


In [ ]:
features_df['SPY Returns

In [76]:
#y = features_df['y']
y = y.iloc[1:]
y

2022-08-29   -1
2022-08-30   -1
2022-08-31   -1
2022-09-01    0
2022-09-02   -1
             ..
2024-02-20   -1
2024-02-21    0
2024-02-22    1
2024-02-23    0
2024-02-26    0
Name: y, Length: 375, dtype: int64

In [51]:
X

,EWC Close,EWJ Close,EWW Close,EZU Close,EWJ Returns,EWW Returns,EZU Returns,EWC Returns,SPY Returns
2022-08-29,34.96,54.44,46.54,35.59,-0.024373,-0.022885,-0.033931,-0.024009,-0.033849
2022-08-30,34.98,53.89,45.92,35.71,-0.010103,-0.013322,0.003372,0.000572,-0.006612
2022-08-31,34.13,53.83,45.17,35.60,-0.001113,-0.016333,-0.003080,-0.024300,-0.010978
2022-09-01,33.69,53.59,44.25,35.33,-0.004458,-0.020368,-0.007584,-0.012892,-0.007609
2022-09-02,33.36,52.95,44.68,34.80,-0.011943,0.009718,-0.015001,-0.009795,0.003138
...,...,...,...,...,...,...,...,...,...
2024-02-20,36.68,67.63,67.30,47.78,-0.000148,-0.004143,-0.002089,-0.000545,-0.004980
2024-02-21,36.54,67.93,67.31,48.06,0.004436,0.000149,0.005860,-0.003817,-0.005505
2024-02-22,36.49,67.79,67.12,48.31,-0.002061,-0.002823,0.005202,-0.001368,0.000906
2024-02-23,36.84,68.90,67.20,48.98,0.016374,0.001192,0.013869,0.009592,0.020695


In [72]:
# import libraries (potentially remove this block of code if redundant)
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
#symbol_returns_lag['y'] = 0
#symbol_returns_lag.loc[x_concat['SPY Returns'] > .002,'y'] = 1
#symbol_returns_lag

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [78]:
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [82]:
number_inputs = 4
number_hidden_nodes = 3
neuron = Sequential()
neuron.add(Dense(units=number_hidden_nodes, input_dim=number_inputs, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
neuron.add(Dense(1, activation="Softmax"))

In [83]:
neuron.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 3)                 15        
                                                                 
 dense_8 (Dense)             (None, 1)                 4         
                                                                 
Total params: 19 (76.00 Byte)
Trainable params: 19 (76.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [84]:
# Compile the Sequential model
neuron.compile(loss="CategoricalCrossentropy", optimizer="adam", metrics=["accuracy"])

In [85]:
# Fit the model using 100 epochs and the training data
model = neuron.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
9/9 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2989
Epoch 2/100
9/9 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2989
Epoch 3/100
9/9 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2989
Epoch 4/100
9/9 [==============================] - 0s 815us/step - loss: 0.0000e+00 - accuracy: 0.2989
Epoch 5/100
9/9 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2989
Epoch 6/100
9/9 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2989
Epoch 7/100
9/9 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2989
Epoch 8/100
9/9 [==============================] - 0s 876us/step - loss: 0.0000e+00 - accuracy: 0.2989
Epoch 9/100
9/9 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.2989
Epoch 10/100
9/9 [==============================] - 0s 876us/step - loss: 0.0000e+00 - 

In [ ]:
# Create a DataFrame using the model history and an index parameter
model_plot = pd.DataFrame(model.history, index=range(1, len(model.history["loss"]) + 1))
# Vizualize the model plot where the y-axis displays the loss metric
model_plot.plot(y="loss")

In [ ]:
# Vizualize the model plot where the y-axis displays the accuracy metric
model_plot.plot(y="accuracy")

In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = neuron.evaluate(X_test_scaled, y_test, verbose=2)
# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# *Side step*
ticker_close[[
    'SPY Close','EWA Close','EWC Close','EWJ Close','EWU Close',
    'EWZ Close','EWW Close', 'EZU Close','IWM Close','MCHI Close']].corr()

In [ ]:
# *sorted data*
c = ticker_close[[
    'SPY Close','EWA Close','EWC Close','EWJ Close','EWU Close',
    'EWZ Close','EWW Close', 'EZU Close','IWM Close','MCHI Close']].corr().abs()
s = c.unstack()
sorted = s.sort_values(kind="quicksort")
print(sorted)

In [ ]:
print(f'The strongest correlation with SPY is EWJ at approximately +.933')

In [ ]:
# CONSTRUCT NEURAL NETWORK
# STEP 1: Ascribe greater weights to ETFs with higher correlation with SPY.
# Use weighted data to predict whether SPY will go up, down, or remain "flat".
# (i.e. EWW has a correlation of .85 with SPY; therefore, if EWW closing price goes up, 
# SPY close has a high chance of going up as well...)
#---------------------------------------------------------------------------------------
# STEP 2: Once neural network has been trained, generate random data to test model for 
# real-world accuracy.
#---------------------------------------------------------------------------------------
# Define features/ (close) pct chng add to features


